In [1]:
import tensorflow as tf
import glob


In [5]:
image_filenames = glob.glob('.\\Images\\n02*\\*.jpg')

In [8]:
image_filenames[0].split('\\')

['.', 'Images', 'n02085620-Chihuahua', 'n02085620_10074.jpg']

In [9]:
from collections import defaultdict
from itertools import groupby

In [10]:
train_data = defaultdict(list)
test_data = defaultdict(list)

In [11]:
image_filename_with_breed = map(lambda filename: (filename.split('\\')[2], filename), image_filenames)

In [13]:
for dog_breed, breed_images in groupby(image_filename_with_breed, lambda x:x[0]):
    for i, breed_image in enumerate(breed_images):
        if i%5==0:
            test_data[dog_breed].append(breed_image[1])
        else:
            train_data[dog_breed].append(breed_image[1])
    
    # train_count = len(train_data[dog_breed])
    # test_count = len(test_data[dog_breed])
    
    # assert round(test_count/(test_count+train_count),2) > 0.18
    

In [22]:
sess = tf.Session()

ERROR! Session/line number was not unique in database. History logging moved to new session 234


In [23]:
def write_records_file(dataset, record_location):
    writer = None
    
    current_index=0
    for breed, breed_images in dataset.items():
        for breed_image in breed_images:
            if current_index%100 == 0:
                if writer:
                    writer.close()
                
                record_filename = '{record_location}-{current_index}.tfrecords'.format(record_location=record_location, current_index=current_index)
                writer = tf.python_io.TFRecordWriter(path=record_filename)
            
            current_index +=1
            image_file = tf.read_file(breed_image)
            try:
                image = tf.image.decode_jpeg(image_file)
            except: 
                print(breed_image)
                continue
                
            grayscale_image = tf.image.rgb_to_grayscale(image)
            resized_image = tf.image.resize_images(grayscale_image, 250, 151)
            
            image_bytes = sess.run(tf.cast(resized_image),tf.uint8).tobytes()
            
            image_label = breed.encode('utf-8')
            
            example = tf.train.Example(features = tf.train.Features(feature = {
                'label': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_label])) ,
                'image': tf.train.Feature(bytes_list = tf.train.BytesList(value=[image_bytes]))
            }))
            
            writer.write(example.SerializeToString())
        
    writer.close()
                

In [ ]:
write_records_file(train_data, '\\train_image')

In [ ]:
write_records_file(test_data, '\\test_image')

In [ ]:
filename_queue = tf.train.string_input_producer(
tf.train.match_filenames_once("./output/training-images/*.tfrecords"))
reader = tf.TFRecordReader()
_, serialized = reader.read(filename_queue)
features = tf.parse_single_example(
serialized,
features={
'label': tf.FixedLenFeature([], tf.string),
'image': tf.FixedLenFeature([], tf.string),
})
record_image = tf.decode_raw(features['image'], tf.uint8)
# Changing the image into this shape helps train and visualize the output by converting it to
# be organized like an image.
image = tf.reshape(record_image, [250, 151, 1])
label = tf.cast(features['label'], tf.string)
min_after_dequeue = 10
batch_size = 3
capacity = min_after_dequeue + 3 * batch_size
image_batch, label_batch = tf.train.shuffle_batch(
[image, label], batch_size=batch_size, capacity=capacity, min_after_dequeue=min_after_dequeue)